# **DenseNet-169 for multi-class classification of CXR images**

### Objectives

The aim of this notebook is to train a selected pretrained network [TBD] on the Chest X-ray (Pneumonia) data set sourced from Kaggle. The images are composed of CXR scans representing normal lungs, bacterial pneumonia, and viral pneumonia.

A **classification head** will be built on top of the chosen network, in which various architectures are developed and evaluated. Several preprocessing techniques are applied, each of which will be evaluated based on the model performance when trained on them. Once trained and evaluated, each version of the network will be further evaluated with the **GRAD-CAM** framework to visualise the regions of importance in the images.

The experiment tracking will be conducted with the *Weights and Biases* platform. 

### Machine Configurations

`GPU` : NVIDIA GeForce RTX 4090;
`CPU` : AMD Ryzen 7 3700X 8-Core Processor;
`VRAM` : 24.0 GB

### Load all required libraries:

In [2]:
import os
import matplotlib.pyplot as plt

import tensorflow as tf

# Suppress TensorFlow messages:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

print("TensorFlow version : ", tf.__version__)  # check TensorFlow version
gpu = tf.config.list_physical_devices("GPU")    # check if TensorFlow is using the GPU
print(gpu)

# Enable memory growth for GPU:
try:
   tf.config.experimental.set_memory_growth(gpu[0], True)
   print("Memory growth set.")
except:
    print("GPU runtime already initialised.")

# Configure notebook name for WandB:
%env "WANDB_NOTEBOOK_NAME" "PCXR-evaluating-pretrained-cnn-model"
import wandb
from wandb.keras import WandbCallback
wandb.login()

TensorFlow version :  2.10.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Configure experiment:

In [ ]:
sweep_config = {
    "method": "bayes"
    }

In [ ]:
metric = {
    "name": "val_loss",
    "goal": "minimize"   
    }

sweep_config["metric"] = metric

In [ ]:
param_dict = {
    "batch_size": {
        "values": [16, 32, 64]
        },
    "optimiser": {
        "values": ["adam", "sgd"]
        },
    "fc_layer_size": {
        "values": [128, 256, 512]
        },
    "dropout": {
        "values": [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = param_dict

In [ ]:
param_dict.update({
    "epochs": {
        'value': 10}
    })

In [ ]:
param_dict.update({
    'learning_rate': {
        "distribution": "uniform",               # uniform distribution:
        "min": 0.0001,
        "max": 0.005
      }
    })

In [ ]:
config = dict(
    learning_rate = 0.0001,
    architecture = "DENSENET-169",
    dataset      = "Chest X-Ray Images (Pneumonia)",
    class_names  = ["normal", "bacteria", "virus"],
    num_classes  = 3,
    val_split    = 0.2,
    init_epochs  = 10,  # initial epochs
    fine_epochs  = 30,  # fine-tuning epochs
    batch_size   = 32,
    image_size   = (300, 300),
    dropout_rate = 0.2,
    activation   = "softmax"
)

# Start wandb run and instantiate project
wandb.init(
    project="PCXR-project", 
    name="custom-logger-test",
    notes="testing custom metrics through callback",
    config=config
)